## Template Matching and Edge Detection Feature Sets

This notebook includes functions and the pipeline for pulling edges and template matching statistics as features

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import cv2
from glob import glob
import utils

# Data paths
DATA_PATH = None  # Will be set by setup_data function

2025-03-24 17:35:05.148491: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 17:35:05.166342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-24 17:35:05.188075: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-24 17:35:05.192597: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 17:35:05.202714: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def setup_data():
    """Set up the data path based on environment."""
    global DATA_PATH
    
    # Check if we're in Colab
    try:
        import google.colab
        from google.colab import drive
        
        # Mount Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')
        
        # Ask for path to project folder
        print("Enter path to the 281_final_project_data folder in Drive (e.g., /content/drive/MyDrive/281_final_project_data):")
        project_path = input()
        
        # Set the data path to the basic folder within the project
        DATA_PATH = os.path.join(project_path, "basic")
            
    except ImportError:
        # Not in Colab, download the data if needed
        download_path = "./281_final_project_data"
        
        # Check if data already exists
        if os.path.exists(download_path) and os.path.exists(os.path.join(download_path, "basic")):
            print(f"Using existing data in {download_path}/basic")
        else:
            print("Download data 281_final_project_data folder from Google Drive.")
        
        # Set the data path to the basic folder within the downloaded project
        DATA_PATH = os.path.join(download_path, "basic")


In [3]:
# Load images function
def load_data(directory):
    
    # First, ensure the data is available
    setup_data()
    path_to_data = DATA_PATH
    
    # load text file with image labels as a dictionary
    labels = pd.read_csv(os.path.join(path_to_data, "EmoLabel/list_patition_label.txt"), sep=" ", header=None)
    labels = dict(zip(labels[0], labels[1]))
    
    # update path_to_data
    path_to_data = os.path.join(path_to_data, "Image", directory)

    train_img = []
    train_labels = []
    test_img = []
    test_labels = []
    
    for file in os.listdir(path_to_data):
        image_path = os.path.join(path_to_data, file)
        image = load_img(image_path)
        img_arr = img_to_array(image, dtype=int)
        if directory == "aligned":
            label = labels[file.replace("_aligned", "")]
        else:
            label = labels[file]
        if "train" in file:
            train_img.append(img_arr)
            train_labels.append(label)
        else:
            test_img.append(img_arr)
            test_labels.append(label)

    train_labels = np.array(train_labels, dtype=int)
    test_labels = np.array(test_labels, dtype=int)
    
    if directory == "aligned":
        train_img = np.array(train_img)
        test_img = np.array(test_img)

        # Apply random shuffling to training examples.
        np.random.seed(0)
        indices = np.arange(train_img.shape[0])
        shuffled_indices = np.random.permutation(indices)
        train_img = train_img[shuffled_indices]
        train_labels = train_labels[shuffled_indices]

    return train_img, train_labels, test_img, test_labels

In [ ]:
# Load label names
label_names_dict = {1: 'surprise', 2: 'fear', 3: 'disgust', 4: 'happiness', 5: 'sadness', 6: 'anger', 7: 'neutral'}

# Images are loaded as ints from 0 to 255
# Load training images and labels (original)
# Note: Loads images as a regular list since images are all different sizes. Labels are numpy array. 
# If loading originals, they will require further processing and shuffling. 
#train_img_org, train_labels_org, test_img_org, test_labels_org = load_data(DATA_PATH, "original")

# Load training images and labels (aligned)
# Converts images to numpy array since images are 100x100 and shuffles them. Labels are numpy array. 
train_img_aligned, train_labels_aligned, test_img_aligned, test_labels_aligned = load_data("aligned")


### Adding Canny Edges to the Images

Canny edges can be added as a channel into the images (shape will then become (H, W, 4) instead of (H, W, 3)) and added as an input into the model, or be used in a pipeline to identify contours of facial landmarks (such as eyes, mouth, nose, etc), or sillhouettes, which can be included as feature statistics into the model.

The below function adds the canny filters into the image as an extra channel and outputs the manipulated image into a designated output folder. we can adjust this to output each image individually or some other format for our pipeline, depending on how it fits with the other feature extraction steps. 

### Template Matching statistics as Selected Features
One way to use template matching for feature engineering would be to have templates for each facial expression (e.g. raised eyebrows and open mouth for suprise, smile and thinner eyes for happy, etc). The below function gathers similarity metrics between an image and a template and could be implemented into a pipeline for us to gather these summary statistics across all facial expression templates. E.g. min_val, max_val, avg_score, std_score will be gathered for each facial expression and included in our feature set. 

One way of getting these templates is to use the training images and get an average face for each emotion class, then apply the canny detection pipeline to each template. This may add more information than a simple smiley face or frowning face. But if we do this should pay close attention to the test statistics as the templates could be overfit to the training data. We should also inspect those average faces to see if this seems feasible


In [ ]:


# processing all images and outputing list of canny edges
def process_images_with_canny(images, lower_threshold = 50, upper_threshold = 150): #suggested thresholds for detecting edges with canny
    """ Converts images to grayscale, enhances contrast, and applies Canny edge detection. outputs list of np.arrays"""

    processed_images = []
       
    for img in images:
        # grayscaling
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # equalizing histogram (contrast)
        img_eq = cv2.equalizeHist(img)
        
        # applying canny edge detection
        edges = cv2.Canny(img_eq, lower_threshold, upper_threshold)
        
        # appending to output list
        processed_images.append(edges)

    return processed_images
 
 
 
# getting an average face for each expression from our training data and applying canny filter 

def create_average_template(images, labels):
    """Creates an average face for all images for each emotion class, then outputting into a dict."""

    emotion_dict = {}
    unique_labels = set(labels)

    for lab in unique_labels:
        # Select all images with the current label
        float_images = [
            cv2.equalizeHist(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).astype(np.float32)
            for img, label in zip(images, labels) if label == lab
        ]
    
        #getting average image of list
        avg_img = np.mean(float_images, axis=0)
    
        #applying canny filter
        avg_img = cv2.Canny(avg_img, 50, 150)
        
        # appending to dict
        emotion_dict[lab] = avg_img        
        

    return emotion_dict





# getting similarity statistics and outputing a vector of features for each emotion template 
def extract_expression_match_features_batch(images, labels, emotion_templates):
    """
    Takes a list of images and labels, and a dict of preprocessed emotion templates (as np.arrays).
    Returns:
        - A list of feature vectors (one per image).
        - A list of corresponding labels.
    """
    feature_matrix = []
    output_labels = []

    for img, label in zip(images, labels):
        # grayscale and equalizing imgs
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.equalizeHist(img)
        #applying canny edges
        img_edges = cv2.Canny(img, 50, 150)

        feature_vector = []

        #getting similarity metrics for each emotion template and adding as features 
        for emotion, template in emotion_templates.items():

            result = cv2.matchTemplate(img_edges, template, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            avg_score = np.mean(result)
            std_score = np.std(result)

            feature_vector.extend([max_val, avg_score, std_score])

        feature_matrix.append(np.array(feature_vector))
        output_labels.append(label)

    return feature_matrix, output_labels



### Applying Functions to pipeline

In [ ]:
# processing and outputing edges vector 
edges_features_2d = process_images_with_canny(train_img_aligned)
edges_features_1d = edges_features_2d.reshape(train_img_aligned.shape[0], -1) # flattening to just 1d feature vector for each image 


# creating emotion templates
templates_dict = create_average_template(train_img_aligned, train_labels_aligned)


# gathering similarity metrics to add to the feature set
template_match_features = extract_expression_match_features_batch(train_img_aligned, train_labels_aligned, templates_dict)


# final edges and template matching feature set
edge_templ_features = np.concatenate([edges_features_1d, template_match_features], axis=1)

